In [ ]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('script')
from script import dbconn
pgconn = dbconn.db_connection_psycopg()

In [ ]:
# Fectching data from the postgreSql database and put the value on raw_df
raw_df = dbconn.db_read_table_psycopg(pgconn,'xdr_data')

# User Overview analysis

In [ ]:
# Identifying the top 10 handsets used by the customers.
# Count handset usage
handset_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results
sorted_handsets = handset_counts.sort_values(by='Count', ascending=False)

# Select the top 10 handsets
top_10_handsets = sorted_handsets.head(10)

# Print the top 10 handsets
print(top_10_handsets)

In [ ]:
# Identifying Top 3 handset manufacturers
manufacturer_counts = raw_df['Handset Manufacturer'].value_counts().reset_index()
manufacturer_counts.columns = ['Handset Manufacturer', 'Count']

# Sort the results
sorted_manufacturers = manufacturer_counts.sort_values(by='Count', ascending=False)

# Select the top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturers.head(3)

# Print the top 3 handset manufacturers
print(top_3_manufacturers)

In [ ]:
# Identify the top 5 handsets per top 3 handset manufacturer
# Count handset manufacturers and types
manufacturer_type_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results within each manufacturer
sorted_manufacturer_types = manufacturer_type_counts.groupby('Handset Manufacturer').apply(lambda x: x.nlargest(5, 'Count')).reset_index(drop=True)

# Print the top 5 handsets per top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturer_types['Handset Manufacturer'].unique()[:3]

for manufacturer in top_3_manufacturers:
    print(f"Top 5 handsets for {manufacturer}:")
    manufacturer_handsets = sorted_manufacturer_types[sorted_manufacturer_types['Handset Manufacturer'] == manufacturer]
    print(manufacturer_handsets)
    print()

In [ ]:
# Identifying users’ behaviour on those Applications / Social Media, Google, Email, Youtube, Netflix, Gaming, Other.   

In [ ]:
# Aggregate the number of xDR sessions per user
sessions_per_user = raw_df.groupby('MSISDN/Number')['Bearer Id'].count()
print(sessions_per_user)

In [ ]:
# Aggregate the session duration per user
session_duration_per_user = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Display the result
print(session_duration_per_user)

In [ ]:
# Aggregate the total download (Dl) and upload data per user
total_data_per_user = raw_df.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum()

# Display the result
print(total_data_per_user)

In [ ]:
# Create a list of application columns
applications = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Aggregate the total data volume per user and application
total_data_per_user_app = raw_df.groupby('MSISDN/Number')[[col + ' DL (Bytes)' for col in applications] + [col + ' UL (Bytes)' for col in applications]].sum()

# Display the result
print(total_data_per_user_app)

In [ ]:
# Explanatory Data Analysis | EDA
# Treat all missing values and outliers in the dataset by replacing by the mean of the corresponding column.

In [ ]:
raw_df.isna().sum()

In [ ]:
#percent of missing data
def percent_missing(df):
    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    percentageMissing = (totalMissing / totalCells) * 100

    print("The dataset contains", round(percentageMissing, 2), "%", "missing values.")

percent_missing(raw_df)

In [ ]:
# Replace missing values with column mean for numeric columns
numeric_columns = raw_df.select_dtypes(include=[np.number]).columns
raw_df[numeric_columns] = raw_df[numeric_columns].fillna(raw_df[numeric_columns].mean())

# Replace missing values with column mode for non-numeric columns
non_numeric_columns = raw_df.select_dtypes(exclude=[np.number]).columns
raw_df[non_numeric_columns] = raw_df[non_numeric_columns].fillna(raw_df[non_numeric_columns].mode().iloc[0])

# Identify and replace outliers with column mean for numeric columns
for col in numeric_columns:
    z_scores = (raw_df[col] - raw_df[col].mean()) / raw_df[col].std()
    outliers = (z_scores > 3) | (z_scores < -3)
    raw_df.loc[outliers, col] = raw_df[col].mean()

# Verify missing values and outliers have been treated
missing_values_after_treatment = raw_df.isnull().sum()
print("Missing Values After Treatment:\n", missing_values_after_treatment)

In [ ]:
raw_df.isna().sum()

In [ ]:
#Solving The rest of missing values
def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]

raw_df['Start'] = fix_missing_ffill(raw_df, 'Start')
raw_df['End'] = fix_missing_ffill(raw_df, 'End')
raw_df['Last Location Name'] = fix_missing_ffill(raw_df, 'Last Location Name')

missing_values = raw_df.isna().sum()
print(missing_values)

In [ ]:
# Analyze the basic metrics (mean, median, etc) from the dataset
raw_df.describe()

In [ ]:
# Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable.

quantitative_variables = []

# Iterate over each column in the dataset to find the quantitative variable
for column in raw_df.columns:   
    if raw_df[column].dtype in [int, float]:
        quantitative_variables.append(column)

# Solution: By calculating the range or Difference b/n max and min value in each variable
for column_name in quantitative_variables:
    column_data = raw_df[column_name]
    data_range = column_data.max() - column_data.min()
    print("Range of", column_name, ":", data_range)

In [ ]:
# Conduct a Graphical Univariate Analysis by identifying the most suitable plotting options for each variable and interpret your findings.
# column_name = 'Avg RTT DL (ms)'
clean_Data = raw_df.dropna()
column_names = clean_Data.columns

for column_name in column_names:
    column_data = clean_Data[column_name]    
    plt.hist(column_data, bins=10)
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + column_name)
    plt.show()

In [ ]:
# Variable transformations
# Segment the users into top five decile classes based on the total duration for all sessions and compute the total data (DL+UL) per decile class. 

# Calculate the total duration for all sessions for each user
user_total_duration = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Segment users into decile classes
user_deciles = pd.qcut(user_total_duration, q=10, labels=False, duplicates='drop')

# Compute the total data (DL+UL) per decile class
data_per_decile = raw_df.groupby(user_deciles)[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()

In [ ]:
# Correlation Analysis – compute a correlation matrix for the following variables and interpret your findings: Social Media data, Google data, Email data, Youtube data, Netflix data, Gaming data, Other data 
# Select the columns for correlation analysis
columns = [
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)'
]

# Create a subset dataframe with the selected columns
subset_df = raw_df[columns]

# Compute the correlation matrix
correlation_matrix = subset_df.corr()

# Display the correlation matrix
print(correlation_matrix)

#  User Engagement analysis

In [ ]:
# Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric

# Aggregate metrics per customer
aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Rename the columns for better readability
aggregated_data.rename(columns={
    'Bearer Id': 'Session Frequency',
    'Dur. (ms)': 'Session Duration',
    'Total UL (Bytes)': 'Total Upload Traffic',
    'Total DL (Bytes)': 'Total Download Traffic'
}, inplace=True)

# Report the top 10 customers per engagement metric
top_10_frequency = aggregated_data.nlargest(10, 'Session Frequency')
top_10_duration = aggregated_data.nlargest(10, 'Session Duration')
top_10_upload_traffic = aggregated_data.nlargest(10, 'Total Upload Traffic')
top_10_download_traffic = aggregated_data.nlargest(10, 'Total Download Traffic')

# Display the results
print("Top 10 customers by Session Frequency:")
print(top_10_frequency)

print("\nTop 10 customers by Session Duration:")
print(top_10_duration)

print("\nTop 10 customers by Total Upload Traffic:")
print(top_10_upload_traffic)

print("\nTop 10 customers by Total Download Traffic:")
print(top_10_download_traffic)

In [ ]:
# Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 
# Aggregate metrics per customer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Normalize the engagement metrics
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(aggregated_data.iloc[:, 1:])  # Exclude customer ID

# Run k-means clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(normalized_data)

# Add the cluster labels to the aggregated data
aggregated_data['Cluster'] = kmeans.labels_

# Report the top customers per cluster
top_customers_per_cluster = []
for i in range(k):
    cluster_customers = aggregated_data[aggregated_data['Cluster'] == i].nlargest(10, 'Bearer Id')
    top_customers_per_cluster.append(cluster_customers)

# Display the results
for i, cluster_customers in enumerate(top_customers_per_cluster):
    print(f"\nTop 10 customers in Cluster {i+1}:")
    print(cluster_customers)

In [ ]:
# Compute the minimum, maximum, average & total non-normalized metrics for each cluster. 
# Interpret your results visually with accompanying text explaining your findings.

# Aggregate metrics per customer
aggregated_data = raw_df.groupby('MSISDN/Number').agg({
    'Bearer Id': 'nunique',  # Session frequency
    'Dur. (ms)': 'sum',  # Session duration
    'Total UL (Bytes)': 'sum',  # Upload traffic
    'Total DL (Bytes)': 'sum'  # Download traffic
}).reset_index()

# Normalize the engagement metrics
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(aggregated_data.iloc[:, 1:])  # Exclude customer ID

# Run k-means clustering
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(normalized_data)

# Add the cluster labels to the aggregated data
aggregated_data['Cluster'] = kmeans.labels_

# Compute non-normalized metrics for each cluster
cluster_metrics = aggregated_data.groupby('Cluster').agg({
    'Bearer Id': ['min', 'max', 'mean', 'sum'],  # Session frequency
    'Dur. (ms)': ['min', 'max', 'mean', 'sum'],  # Session duration
    'Total UL (Bytes)': ['min', 'max', 'mean', 'sum'],  # Upload traffic
    'Total DL (Bytes)': ['min', 'max', 'mean', 'sum']  # Download traffic
})

# Display the results
print("Non-normalized metrics for each cluster:")
print(cluster_metrics)

In [ ]:
# Aggregate user total traffic per application and derive the top 10 most engaged users per application

# Extract the relevant columns for application traffic
app_columns = ['MSISDN/Number', 'Social Media DL (Bytes)', 'Google DL (Bytes)', 
               'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 
               'Gaming DL (Bytes)', 'Other DL (Bytes)']
app_traffic = raw_df[app_columns].copy()

# Rename the application columns for easier processing
app_traffic.columns = ['MSISDN/Number', 'Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Melt the dataframe to combine all application columns into a single 'Application' column
app_traffic = app_traffic.melt(id_vars='MSISDN/Number', var_name='Application', value_name='Total Traffic')

# Aggregate total traffic per application for each user
app_traffic = app_traffic.groupby(['MSISDN/Number', 'Application'])['Total Traffic'].sum().reset_index()

# Derive the top 10 most engaged users per application
top_users_per_app = []
unique_apps = app_traffic['Application'].unique()

for app in unique_apps:
    top_users = app_traffic[app_traffic['Application'] == app].nlargest(10, 'Total Traffic')
    top_users_per_app.append(top_users)

# Display the results
for i, app in enumerate(unique_apps):
    print(f"\nTop 10 most engaged users for Application '{app}':")
    print(top_users_per_app[i])

In [ ]:
# Plot the top 3 most used applications using appropriate charts. 

# Extract the relevant columns for application traffic
app_columns = ['MSISDN/Number', 'Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
app_traffic = raw_df[app_columns].copy()

# Rename the application columns for easier processing
app_traffic.columns = ['MSISDN/Number', 'Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Melt the dataframe to combine all application columns into a single 'Application' column
app_traffic = app_traffic.melt(id_vars='MSISDN/Number', var_name='Application', value_name='Total Traffic')

# Aggregate total traffic per application
app_traffic = app_traffic.groupby('Application')['Total Traffic'].sum().reset_index()

# Sort applications by total traffic in descending order
app_traffic = app_traffic.sort_values('Total Traffic', ascending=False)

# Select the top 3 most used applications
top_3_apps = app_traffic.head(3)

# Plot the top 3 most used applications
plt.figure(figsize=(8, 6))
plt.bar(top_3_apps['Application'], top_3_apps['Total Traffic'])
plt.xlabel('Application')
plt.ylabel('Total Traffic')
plt.title('Top 3 Most Used Applications')
plt.show()

In [ ]:
# Using k-means clustering algorithm, group users in k engagement clusters based on the engagement metrics: 
# What is the optimized value of k (use elbow method for this)?

# Experience Analytics

In [ ]:
# Aggregate average TCP retransmission per customer
average_tcp_retransmission = raw_df.groupby('MSISDN/Number')['TCP DL Retrans. Vol (Bytes)'].mean()

# Print the aggregated results
print("Average TCP Retransmission per Customer:")
print(average_tcp_retransmission)

In [ ]:
# Aggregate average RTT per customer
average_rtt = raw_df.groupby('MSISDN/Number')['Avg RTT DL (ms)'].mean()

# Print the aggregated results
print("Average RTT per Customer:")
print(average_rtt)

In [ ]:
# Aggregate handset type per customer
handset_type = raw_df.groupby('MSISDN/Number')['Handset Type'].first()

# Print the aggregated results
print("Aggregate handset type per Customer:")
print(handset_type)

In [ ]:
# Aggregate average throughput per customer
average_throughput = raw_df.groupby('MSISDN/Number')['Avg Bearer TP DL (kbps)'].mean()

# Print the aggregated results
print("Average Throughput per Customer:")
print(average_throughput)

In [ ]:
# Compute & list 10 of the top, bottom and most frequent:

In [ ]:
# Get the top 10 TCP values
top_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].nlargest(10)

# Get the bottom 10 TCP values
bottom_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].nsmallest(10)

# Get the most frequent TCP values
most_frequent_tcp_values = raw_df['TCP DL Retrans. Vol (Bytes)'].value_counts().head(10)

# Print the results
print("Top 10 TCP Values:")
print(top_tcp_values)
print("\nBottom 10 TCP Values:")
print(bottom_tcp_values)
print("\nMost Frequent TCP Values:")
print(most_frequent_tcp_values)

In [ ]:
# Get the top 10 RTT values
top_rtt_values = raw_df['Avg RTT DL (ms)'].nlargest(10)

# Get the bottom 10 RTT values
bottom_rtt_values = raw_df['Avg RTT DL (ms)'].nsmallest(10)

# Get the most frequent RTT values
most_frequent_rtt_values = raw_df['Avg RTT DL (ms)'].value_counts().head(10)

# Print the results
print("Top 10 RTT Values:")
print(top_rtt_values)
print("\nBottom 10 RTT Values:")
print(bottom_rtt_values)
print("\nMost Frequent RTT Values:")
print(most_frequent_rtt_values)

In [ ]:
# Get the top 10 throughput values
top_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].nlargest(10)

# Get the bottom 10 throughput values
bottom_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].nsmallest(10)

# Get the most frequent throughput values
most_frequent_throughput_values = raw_df['Avg Bearer TP DL (kbps)'].value_counts().head(10)

# Print the results
print("Top 10 Throughput Values:")
print(top_throughput_values)
print("\nBottom 10 Throughput Values:")
print(bottom_throughput_values)
print("\nMost Frequent Throughput Values:")
print(most_frequent_throughput_values)

In [ ]:
# Compute the distribution of average throughput per handset type
avg_throughput_distribution = raw_df.groupby('Handset Type')['Avg Bearer TP DL (kbps)'].mean()

# Plot the distribution
plt.figure(figsize=(12, 6))
avg_throughput_distribution.plot(kind='bar')
plt.xlabel('Handset Type')
plt.ylabel('Average Throughput (kbps)')
plt.title('Distribution of Average Throughput per Handset Type')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Compute the average TCP retransmission view per handset type
avg_tcp_retransmission = raw_df.groupby('Handset Type')['TCP DL Retrans. Vol (Bytes)'].mean()

# Plot the average TCP retransmission view per handset type
plt.figure(figsize=(12, 6))
avg_tcp_retransmission.plot(kind='bar')
plt.xlabel('Handset Type')
plt.ylabel('Average TCP Retransmission')
plt.title('Average TCP Retransmission View per Handset Type')
plt.xticks(rotation=90)
plt.show()

In [53]:
from sklearn.cluster import KMeans

# Select the relevant columns for clustering
data = raw_df[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']]

# Perform k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(data)

# Add cluster labels to the dataset
raw_df['Cluster'] = kmeans.labels_

# Describe each cluster
cluster_descriptions = raw_df.groupby('Cluster').agg({
    'Avg RTT DL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'TCP DL Retrans. Vol (Bytes)': 'mean'
})

# Print the cluster descriptions
print("Cluster Descriptions:")
print(cluster_descriptions)

c:\Users\o876\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster Descriptions:
         Avg RTT DL (ms)  Avg Bearer TP DL (kbps)  TCP DL Retrans. Vol (Bytes)
Cluster                                                                       
0              72.172680              3086.689841                 2.157774e+07
1             107.469523             25438.156432                 1.506082e+06
2             140.401249             35630.835801                 1.873654e+08
